## This is a general manual for the steps in analysis of Fleckvieh population

Based on the manualscript created by Jasna and follow-up steps

### Training data set:
-	CPFV = Case population Fleckvieh (recent Fleckvieh animals, born after 2010) --> around 1300 animals
-	RPFV = Reference population Fleckvieh (25 oldest Fleckvieh animals, born back to 1971)
-	RPRH = Reference population Red Holstein (25 oldest Red Holstein animals, born before 1990)
-	VCF-files for all 29 chromosomes (includes more animals than the ones we are interested in)
-	PLINK format .map, .ped and .fam-file (.fam-file includes the ~1350 animals we are interested in)

#### 1.	Step: filter the animal relevant to us (.fam-file) from the vcf-files

In [ ]:
### Install vcftools (Google: vcftools conda):
mamba install -c bioconda vcftools
## or
mamba install -c conda-forge vcftools

### Filter animals – example chromosome 29:

vcftools --keep set1.txt --gzvcf OUT_VCF_BEAGLE4_ALL_AutoSom21b_Chr29.txt.vcf.gz --recode --stdout | gzip -c > Chrom29_set1.vcf.gz
## --keep -->  search for the following samples
## set1.txt --> file with animal IDs that we want to keep --> Command to establish this file (set1.txt): $ awk ‘{print $2}’ *.fam > set1.txt
## --gzvcf --> input file as vcf.gz file
## --stdout | gzip -c --> print output in .gz file with the name 


## Filter animals – for all chromosomes (with for-loop in #sbatch):
vi run_vcftools.sh ### opens window where you have to write the batch-job in

## THE FOLLOWING IS WHAT THE VIM FILE LOOK LIKE##########################

## for our case lrz use the slrum/ in the headnode we can just ignore it
#!/bin/bash
#SBATCH --job-name=vcftools_all_chr
#SBATCH --output=vcftools_all_chr.out
#SBATCH --error=vcftools_all_chr.err
#SBATCH --nodes=1
#SBATCH --tasks-per-node=1
#SBATCH --cpus-per-task=8
#SBATCH --mem=100000
#SBATCH --time=48:00:00
#SBATCH --partition=base

export OMP_NUM_THREADS=8
source ~/.bashrc


## Loop for filtering animals for all 1-29 chromosomes
for z in {1..29}; do
    vcftools --keep set1.txt --gzvcf OUT_VCF_BEAGLE4_ALL_AutoSom21b_Chr${z}.txt.vcf.gz --recode --stdout | gzip -c > Chrom${z}_set_rprh_cpfv.vcf.gz
done


## Running the bash script
./run_vcftools.sh
##OR
sbatch run_vcftools.sh


### SLURM Directives:
These lines starting with #SBATCH are directives for SLURM, a workload manager for running jobs on high-performance computing (HPC) clusters.

#### 1. #!/bin/bash:
This is the shebang line, indicating that the script should be executed with the Bash shell.

#### 2. #SBATCH --job-name=vcftools_all_chr:
This sets the name of the job. In this case, the job is named vcftools_all_chr.

#### 3. #SBATCH --output=vcftools_all_chr.out:
The output of the job (standard output) will be written to a file named vcftools_all_chr.out.

#### 4. #SBATCH --error=vcftools_all_chr.err:
If any errors occur during the job's execution, they will be written to a file named vcftools_all_chr.err.

#### 5. #SBATCH --nodes=1:
This requests one node (a computing unit in a cluster).

#### 6. #SBATCH --tasks-per-node=1:
This requests one task (or process) to run on the node.

#### 7. #SBATCH --cpus-per-task=8:
This specifies that each task should use 8 CPUs (or cores). It means that the task is multi-threaded and will utilize 8 CPUs.

#### 8. #SBATCH --mem=100000:
This requests 100,000 MB (100 GB) of memory for the job.

#### 9. #SBATCH --time=48:00:00:
This requests a maximum runtime of 48 hours for the job. If the job takes longer than 48 hours, SLURM will stop it.

#### 10. #SBATCH --partition=base:
This specifies that the job should run on the "base" partition or queue within the cluster. Different partitions might have different priorities, resources, or runtime limits.

### Job execution
1. export OMP_NUM_THREADS=8:
This sets the environment variable OMP_NUM_THREADS to 8, which is useful for multi-threaded programs. It tells the program (in this case, vcftools) to use 8 threads.

2. source ~/.bashrc:
This sources the user's .bashrc file, which loads environment variables and paths defined in that file.

#### 2.	Step: Establish a .map-file for each chromosome (necessary as input file for the program hap-IBD) – as  batch-job:

In [ ]:
for i in {1..29}; do
    zcat Chrom${i}_set_rprh_cpfv.vcf.gz | awk '$o!~/#/{print $1, $3, $2/1000000, $2}' > Chrom${i}_set_rprh_cpfv.map
done

## zcat decompresses the file Chrom1_set_rprh_cpfv.vcf.gz, Chrom2_set_rprh_cpfv.vcf.gz, etc., up to chromosome 29.

## $o!~/#/: This is a condition to skip lines that start with # (i.e., header lines in the VCF file). 
## It processes only lines that don't begin with # (non-comment lines).

## The output from awk is redirected to a new file called Chrom${i}_set_rprh_cpfv.map. 
## For each chromosome, it creates a separate .map file


In [ ]:
## .map file format like
##Chrom1_set_rprh_cpfv.map as example

1 Hapmap43437-BTA-101873 0.776231 776231
1 ARS-BFGL-NGS-16466 0.90781 907810
1 Hapmap34944-BES1_Contig627_1906 1.03256 1032564
1 BTA-07251-no-rs 1.0735 1073496
1 ARS-BFGL-NGS-98142 1.11039 1110393
1 Hapmap53946-rs29015852 1.15076 1150763
1 BFGL-NGS-114208 1.16705 1167048
1 ARS-BFGL-NGS-66449 1.20483 1204825
1 ARS-BFGL-BAC-32770 1.56654 1566539
1 ARS-BFGL-NGS-65067 1.60494 1604940

#### 3.	Step: run hap-IBD – as batch-job and generate ser_rprh_cpfv files:

In [ ]:
for i in {1..29}; do
    hap-ibd gt=Chrom${i}_set_rprh_cpfv.vcf.gz map=Chrom${i}_set_rprh_cpfv.map out=Chrom${i}_set_rprh_cpfv;
done

##hap-ibd is the program used to infer identity-by-descent (IBD) segments in a population or genomic dataset, based on genotype data


#### 4.	Step: sort the hap-IBD output – as batch-job (with for-loop; for big datasets parallelization is helpful):

In [ ]:
for i in {1..29}; do 
    zcat Chrom${i}_set_rprh_cpfv.ibd.gz | sort -g -k6,6 -k7,7 | gzip -c > Chrom${i}_set_rprh_cpfv_sorted.ibd.gz; 
done

## sort -g --> general numeric sort in numerical order (0,1,…,99,100)
## -k6,6 --> speficify the key column to sort by (column 6 in this case)
## -k7,7 --> if the values on column 6 is the same, sort by column 7

### The file looks like
zcat Chrom1_set_rprh_cpfv_sorted.ibd.gz | head

0359411 2       ASR48555        1       1       776231  5147933 4.372
0359411 2       FV2147  1       1       776231  5147933 4.372
AA0000  1       BB0000  2       1       776231  5147933 4.372
AA0000  1       BC0000  2       1       776231  5147933 4.372
AA0000  1       BD0000  2       1       776231  5147933 4.372
AA0000  1       BD0022  2       1       776231  5147933 4.372
AA0000  1       BE0000  2       1       776231  5147933 4.372
AA0000  1       DA0000  1       1       776231  5147933 4.372
AA0000  1       DA0222  1       1       776231  5147933 4.372
AA0000  1       FA0000  1       1       776231  5147933 4.372


#### 5.	Step: run python script (plot_ibd.py) for all chromosomes – as batch-job:

In [ ]:
for i in {1..29}; do 
    python plot_ibd.py Chrom${i}_set_rprh_cpfv_sorted.ibd.gz rprh_pop.txt cpfv_pop.txt Chrom${i}_set_rhrp_cpfv.map Chrom${i}_set_rprh_cpfv.out; 
done

## plot_ibd.py --> python script
## Chrom${i}_set_rprh_cpfv_sorted.ibd.gz --> input file: ibd_output_sorted
## rprh_pop.txt --> source population: reference population RH
## cpfv_pop.txt --> destination population: case population FV
##Chrom${i}_set_rhrp_cpfv.map --> SNP-map-file
## Chrom${i}_set_rprh_cpfv.out --> output file


## in my case, I used the following commands to run in the project diretory
BASE_DIR="/home/maulik/data/Shared/Maulik/projects/fleckvieh_project"

parallel -j 29 python $BASE_DIR/ibdtool/plot_ibd.py $BASE_DIR/hapibd/set_sorted/Chrom{1}_set_rprh_cpfv_sorted.ibd.gz $BASE_DIR/original_data/source_pop_rh.txt $BASE_DIR/original_data/recent_pop_fv_neu.txt $BASE_DIR/hapibd/map_files/Chrom{1}_set_rprh_cpfv.map $BASE_DIR/jade_test/set_out/Chrom{1}_set_rprh_cpfv.out sum_hap ::: {1..29}
### change the number from 29 to 16 due to the number of avaliable GPUs.







##based on plot_ibd.py, two files generated
## Chrom${i}_num_hap.txt
The output contains SNP positions along with the number of haplotypes overlapping each SNP position
<position> <num_haplo>

## Chrom${i}_sum_anc.txt

The output contains two main pieces of information:

The overall difference between the maximum and minimum SNP coordinates.
The sum of haplotype window lengths for each sample.

<sample> <total_haplotype_length>



#### This is the step to generate how many haplotypes counts are in IBD with RH populations by chromosomes.

#### 6.	Step: add chromosome number to all output files – as batch-job:
For each chromosome i, the script processes the file Chrom${i}_set_rprh_cpfv.out using awk to:

- Add the chromosome number as the first column.

- Extract the first and second columns from the original file.

- It writes the output to a new file Chrom${i}_rprh_cpfv_with_chrom_id.out, where the chromosome ID is included as the first column.

In [ ]:
for i in {1..29}; do 
    awk -v Chrom=$i '{print Chrom, $1, $2}' Chrom${i}_set_rprh_cpfv.out > Chrom${i}_rprh_cpfv_with_chrom_id.out; 
done



#### 7.	Step: merge all the output files of plot_ibd.py and sort them in order to see which chromosome has the highest number of IBDs shared with RH:


In [ ]:
cat Chrom{1..29}_rprh_cpfv_with_chrom_id.out | sort -g -r -k3,3 | head

## The merged_sorted_output file looks like this:
8 107482423 23938
8 107445396 23937
8 108463394 23935
8 108535138 23930
8 107385704 23924
8 107547507 23919
8 108648399 23867
8 108603393 23865
8 108243453 23839
8 108879102 23838

sort: sorts the concatenated output.
-g: Performs a general numerical sort, so numbers are sorted based on their numeric values.
-r: Sorts the data in reverse order, meaning it will sort in descending (largest to smallest) order.
-k3,3: This specifies to sort by the third column. The 3,3 means to sort using only the third column.

head: display the first 10 lines of the sorted data

## PART II

Try to generate `sum_anc.txt` file too but change the sixth argument in `plot_ibd.py` than `sum_hap`

#### Step 1 create input files

In [ ]:
parallel -j 16 python prepare_input.py Chrom{1}_set_rprh_cpfv_sorted.ibd.gz source_pop_rh.txt recent_pop_fv_neu.txt chrom{1} ::: {1..29}


The script prepares input data for each chromosome and make a order to decrease the computation loads

This Python script processes genomic data, specifically focusing on identifying shared Identity-By-Descent (IBD) segments between source and target populations or samples. The script takes several input files, processes them, and outputs a sorted list of IBD segments between specified populations.

#### Rearrange the data:
when the second column is the source data, Specifically, it creates a list (n_line) that holds the values: source sample, source haplotype, target sample, target haplotype, chromosome, start, and end positions (columns 0, 1, 2, 3, 4, 5, 6).


In [ ]:
### generated files look like: chrom7_target_sample_sorted.txt
### RH, RH_hap, FL, FL_hap, chrom, starting snp, ending snp

source_sample,ss_hap,target_sample,ts_hap,chrom,start,end
FV0001,2,0359411,2,7,386109,7467822
DA0222,2,ASR00017,2,7,386109,16345831
DA0000,2,ASR00017,2,7,386109,16462754
HF0619,2,ASR00017,2,7,386109,16462754
HF0079,1,ASR00017,2,7,2546807,7841810
HF0087,2,ASR00017,2,7,2546807,7841810
KI-GT3037,2,ASR00017,2,7,2546807,7841810
RH117,2,ASR00017,2,7,16269013,27046365
EA0000,2,ASR00017,2,7,26940967,40388364


#### Step 2 merge overlap within each sample

In [ ]:
parallel -j 29 python merge_hap.py chrom{1}_target_sample_sorted.txt chrom{1}_pre_hap.csv ::: {1..29}


This step merges overlapping regions in haplotype data within each sample, chromosome by chromosome. Generated file after this step:
`chrom{i}_pre_hap.csv`

animal information and haplotype information are merged and all rh_hap in IBD with fl_hap is listed.

In [ ]:
example: `chrom7_pre_hap.csv`

fl_hap,start,end,rh_hap
ASR23942_2,386109,4505086,CC0000_1:386109-4505086 HF0799_1:386109-4505086 KI-GT3046_2:386109-4505086 RH103_1:386109-4505086 RH105_1:386109-4505086
ASR29342_1,386109,4505086,CC0000_1:386109-4505086 HF0799_1:386109-4505086 KI-GT3046_2:386109-4505086 RH103_1:386109-4505086 RH105_1:386109-4505086
ASR29620_1,386109,4505086,CC0000_1:386109-4505086 HF0799_1:386109-4505086 KI-GT3046_2:386109-4505086 RH103_1:386109-4505086 RH105_1:386109-4505086
FV1897_1,386109,4505086,HF0684_2:386109-4505086 RH150_1:386109-4505086 RH150_2:386109-4505086 RR0001_2:386109-4505086


#### Step 3 Infer haplotype

In [ ]:
parallel -j 29 python generate_hap_info.py chrom{1}_pre_hap.csv chrom{1} ::: {1..29}

This step generates three files for each chromosome:
`chrom1_hap_count_info.txt` --> Counts of source and target haplotypes.
`chrom1_tar_hap_info.txt` --> Information about target haplotypes.
`chrom1_source_hap_info.txt` --> Information about source haplotypes.

from the file `chrom1_pre_hap.csv`

#### Step 4 Generate plot for haplotype frequency

In [ ]:
BASE_DIR="/home/maulik/data/Shared/Maulik/projects/fleckvieh_project"

parallel -j 16 python $BASE_DIR/jade_test/hapibd-pytools/plot_haplo_freq.py $BASE_DIR/jade_test/chrom{1}_tar_hap_info.txt chrom{1}_source_hap_info.txt $BASE_DIR/hapibd/map_files/Chrom{1}_set_rprh_cpfv.map chrom{1} ::: {1..29}


This step generates histogram (Bokeh plot) which counts the total number of FL haplotypes in IBD with RH haplotypes 

#### Step 5 Generate plot

In [ ]:
python $BASE_DIR/jade_test/hapibd-pytools/plot_haplo_stack.py $BASE_DIR/jade_test/chrom1_source_hap_info.txt 151 hap368 chrom1_tar_hap_info.txt 5 chrom1_hap368

The script expects six command-line arguments:

`source_hap_f`: Path to the source haplotype file.

`num_source_hap`: Number of source haplotypes. (maximum 65 animals so usually we set to 5)

`hap_id`: Haplotype ID.

`tar_hap_f`: Path to the target haplotype file.

`num_tar_hap`: Number of target haplotypes. (should not exceed the number in the column chrom{i}_hap_count_info.txt$count_of_tar_hap)

`out_prefix`: Prefix for the output file.


The plot generates an **interactive stacked line plot** using the Bokeh library. The plot visualizes haplotype data, where each line represents a haplotype segment, and the segments are stacked vertically. 

